### Restaurant Review Analyzer

Problem:

Given the vast number of restaurants and the various reviews they receive, create a system that categorizes reviews based on their semantic meaning (e.g., ambiance, food quality, service, value for money).



Skills/Functions Needed:
- **extractReviewEssence()**: Utilize semantic kernel plugins to derive the core sentiment or subject from reviews.
- **categorizeReview()**: Use the semantic kernel skill to determine which category a review belongs to.
- **planReviewAnalysis()**: With the semantic kernel Planner, design a flow to efficiently process and categorize bulk reviews.

In [1]:
!python -m pip install semantic-kernel microsoft-bing-websearch azure-cognitiveservices-search-websearch

Imports and GLOBAL variables

In [2]:
import os
import random
import json
from semantic_kernel.connectors.ai.open_ai import OpenAIChatCompletion
from semantic_kernel.connectors.search_engine import BingConnector
from semantic_kernel.core_skills.web_search_engine_skill import WebSearchEngineSkill
from semantic_kernel.skill_definition import sk_function
from azure.cognitiveservices.search.websearch import WebSearchClient
from azure.cognitiveservices.search.websearch.models import SafeSearch
from azure.core.credentials import AzureKeyCredential

BING_API_KEY = 'b980f25ee9c04792ba93789acab6e10b'
skills_directory = "../skills"


Native functions definition

In [3]:
class GenerateNumberSkill:
    """
    Description: Generate a number between 3-x.
    """

    @sk_function(
        description="Generate a random number between 3-x",
        name="GenerateNumberThreeOrHigher"
    )
    def generate_number_three_or_higher(self, input: str) -> str:
        """
        Generate a number between 3-<input>
        Example:
            "8" => rand(3,8)
        Args:
            input -- The upper limit for the random number generation
        Returns:
            int value
        """
        try:
            return str(random.randint(3, int(input))) 
        except ValueError as e:
            print(f"Invalid input {input}")
            raise e

class SearchTheInternetSkill:
    """
    Description: Search the internet for user {{INPUT}}
    """
    @sk_function(
        description="Search the internet for a given topic",
        name="Search"
    )
    def search(self, input: str) -> str:
        try:
            ENDPOINT = "https://api.bing.microsoft.com"+  "/v7.0/"
            client = WebSearchClient(AzureKeyCredential(BING_API_KEY))
            return json.dump(client.web.search(query=input))
        except ValueError as e:
            print(f"Input: {input}\n Error:{e}")
            raise e
        
class ValidatorSkill:
    """
    Description: Native Function: Validate model outputs
    """
    @sk_function(
        description="Validate JSON",
        name="ValidateJson"
    )
    def validate_json(self, input: str) -> str:
        try:
            parsed_json = json.loads(input)
            return input
        except json.JSONDecodeError as e:
            return f"[ERROR]Invalid JSON generated.[/ERROR]"

Initialize kernel with gpt-3.5-turbo OpenAIChatCoompletion

In [4]:
import semantic_kernel as sk
from semantic_kernel.connectors.ai.open_ai import AzureChatCompletion, OpenAIChatCompletion

kernel = sk.Kernel()

useAzureOpenAI = False

# Configure AI service used by the kernel
if useAzureOpenAI:
    deployment, api_key, endpoint = sk.azure_openai_settings_from_dot_env()
    kernel.add_chat_service("chat_completion", AzureChatCompletion(deployment, endpoint, api_key))
else:
    api_key, org_id = sk.openai_settings_from_dot_env()
    kernel.add_chat_service("chat-gpt", OpenAIChatCompletion("gpt-3.5-turbo", api_key, org_id))

In [6]:
"""sk_prompt ="generate a random json array with at most 10 elements and 5 properties each, each payload about {{$INPUT}}, the output must be complete a parsable json, max lenght 512, only print the output"

random_json = kernel.create_semantic_function(sk_prompt, 'json', 'myrandom', 'generate random json about a topic',512)
res = await random_json.invoke_async("restaurant")
print(res)
"""

'sk_prompt ="generate a random json array with at most 10 elements and 5 properties each, each payload about {{$INPUT}}, the output must be complete a parsable json, max lenght 512, only print the output"\n\nrandom_json = kernel.create_semantic_function(sk_prompt, \'json\', \'myrandom\', \'generate random json about a topic\',512)\nres = await random_json.invoke_async("restaurant")\nprint(res)\n'

In [7]:
"""
validator_skill = kernel.import_skill(ValidatorSkill(), 'ValidatorSkill')
json_validator_function = validator_skill["ValidateJson"]
validation_result = json_validator_function(res.result)
print(validation_result.result)
"""

'\nvalidator_skill = kernel.import_skill(ValidatorSkill(), \'ValidatorSkill\')\njson_validator_function = validator_skill["ValidateJson"]\nvalidation_result = json_validator_function(res.result)\nprint(validation_result.result)\n'

In [10]:

#validator_skill = kernel.import_skill(ValidatorSkill(), 'ValidatorSkill')
newsSkills = kernel.import_semantic_skill_from_directory(skills_directory, "NewsSkill")
newsFunction = newsSkills["NewsGenerator"]
gearFunction = newsSkills["GearFinder"]
#res = await newsFunction.invoke_async("I want to create an article about kite surfing in fuerte ventura, the main keywords is life on the island, the second keywords is type of people interested kite surf that chose to goo to fuerte ventura. The tone fo voice should be engaging and the articole should be writte in italian, the article has to be 100 words")
#resGear = await gearFunction.invoke_async("laptop")
from semantic_kernel.planning.basic_planner import BasicPlanner
planner = BasicPlanner()
newsAsk = "list of best places to take nice pictures in Cappadocia, primary keywords baloons, in italian, 500 words"
newsOutput = await newsFunction.invoke_async(newsAsk)

gearAsk = "read the related_purchases property, take the first item with category = gear, and find the best options on the market for the item, the item you have to use to provide the options for will be in the contained in the related_purchases[*].name property of this: " + newsOutput.result
try:

    plan = await planner.create_plan_async(gearAsk, kernel)
    print(plan.generated_plan)
    result = await planner.execute_plan_async(plan, kernel)
    print(result)
except Exception as e:
    print(e)

"""
turboFunctions = kernel.import_semantic_skill_from_directory(skills_directory, "ReviewSkill")

generate_number_skill = kernel.import_skill(GenerateNumberSkill(), 'GenerateNumberSkill')
search_the_internet_skill = kernel.import_skill(SearchTheInternetSkill(), 'SearchTheInternetSkill')


generateReviewsFunction = turboFunctions["GenerateReviews"]
categorizeReviewsFunction = turboFunctions["Categorizer"]

search_function = search_the_internet_skill['Search']

results = search_function("'can fisher' retaurant barcelona, reviews")
print(results)
connector = BingConnector(BING_API_KEY)
web_skill = kernel.import_skill(WebSearchEngineSkill(connector), "WebSearch")

prompt = "find at most 10 most recent reviews about {{$INPUT}} restaurant in {{$CITY}}"

context = kernel.create_new_context()
context["restaurant"] = "inquiet"
context["city"] = "barcelona"

search_async = web_skill["searchAsync"]
result = await search_async.invoke_async(prompt, context=context)
print(result)
"""

{
    "input": "Best places to take nice pictures in Cappadocia",
    "subtasks": [
        {"function": "NewsSkill.JsonParser"},
        {"function": "NewsSkill.GearFinder", "args": {"input": "Camera equipment for landscape photography"}}
    ]
}
[OUTPUT]
[{
	"name":"Nikon D850",
	"brand":"Nikon",
	"reasoning":"The Nikon D850 is widely regarded as one of the best cameras for landscape photography due to its high resolution (45.7 megapixels), excellent dynamic range, and impressive image quality. It also has a rugged build, weather sealing, and a tilting touchscreen for added convenience."
},
{
	"name":"Sony Alpha a7R IV",
	"brand":"Sony",
	"reasoning":"The Sony Alpha a7R IV is another top choice for landscape photography. It features a high-resolution full-frame sensor (61 megapixels), excellent low-light performance, and advanced autofocus capabilities. It also has in-body image stabilization and a wide range of compatible lenses."
},
{
	"name":"Canon EOS 5DS R",
	"brand":"Canon",
	"

'\nturboFunctions = kernel.import_semantic_skill_from_directory(skills_directory, "ReviewSkill")\n\ngenerate_number_skill = kernel.import_skill(GenerateNumberSkill(), \'GenerateNumberSkill\')\nsearch_the_internet_skill = kernel.import_skill(SearchTheInternetSkill(), \'SearchTheInternetSkill\')\n\n\ngenerateReviewsFunction = turboFunctions["GenerateReviews"]\ncategorizeReviewsFunction = turboFunctions["Categorizer"]\n\nsearch_function = search_the_internet_skill[\'Search\']\n\nresults = search_function("\'can fisher\' retaurant barcelona, reviews")\nprint(results)\nconnector = BingConnector(BING_API_KEY)\nweb_skill = kernel.import_skill(WebSearchEngineSkill(connector), "WebSearch")\n\nprompt = "find at most 10 most recent reviews about {{$INPUT}} restaurant in {{$CITY}}"\n\ncontext = kernel.create_new_context()\ncontext["restaurant"] = "inquiet"\ncontext["city"] = "barcelona"\n\nsearch_async = web_skill["searchAsync"]\nresult = await search_async.invoke_async(prompt, context=context)\np

In [9]:
"""
import os
from web_search_client import WebSearchClient
from web_search_client.models import SafeSearch
from azure.core.credentials import AzureKeyCredential


ENDPOINT = "https://api.bing.microsoft.com"+  "/v7.0/"
client = WebSearchClient(AzureKeyCredential(BING_API_KEY))
web_data = client.web.search(query="xbox")
print("Searched for Query# \" Xbox \"")
"""

NameError: name 'search' is not defined

In [10]:
"""
generate_number_three_or_higher = generate_number_skill["GenerateNumberThreeOrHigher"]
number_result = generate_number_three_or_higher(6)
print(number_result)
"""

NameError: name 'generate_number_skill' is not defined

In [ ]:
reviews = generateReviewsFunction("italian")
print(reviews.result)

In [ ]:
print(reviews.result)

cateogirzed = categorizeReviewsFunction(reviews.result)

In [ ]:
print(cateogirzed)